In [5]:
pip install kaggle

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

/Users/vishesh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


100%|██████████| 80.9M/80.9M [01:07<00:00, 1.25MB/s]

Extracting files...


Path to dataset files: /Users/vishesh/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2


In [10]:
from zipfile import ZipFile
dataset='/Users/vishesh/Downloads/One8/training.1600000.processed.noemoticon.csv.zip'
with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print("The dataset is extracted")

The dataset is extracted


In [13]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Download stopwords if not already downloaded
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishesh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
#Data preprocessing
import pandas as pd

file_path = '/Users/vishesh/Downloads/One8/training.1600000.processed.noemoticon.csv'

try:
    twitter_data = pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')
    print("File loaded successfully!")
except Exception as e:
    print("Error:", e)


File loaded successfully!


In [23]:
twitter_data.shape

(1599999, 6)

In [24]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [26]:
#nameing the columns and reading the dataset again

column_name= ['target','id','data','flag','user','text']
twitter_data = pd.read_csv(file_path, encoding='latin1',names=column_name, on_bad_lines='skip')


In [27]:
twitter_data.shape

(1600000, 6)

In [28]:
twitter_data.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [31]:
# counting the number of mising values
twitter_data.isnull().sum()

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [32]:
#checking distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [55]:
# Ensure the column type is integer
twitter_data["target"] = twitter_data["target"].astype(int)

# Replace '4' with '1'
twitter_data["target"] = twitter_data["target"].replace(4, 1)

# Verify replacement
print(twitter_data["target"].value_counts())  # Should show only 0s and 1s


target
0    800000
1    800000
Name: count, dtype: int64


In [ ]:
#Stemming is the process of reducing a word to its root word
# example: actor,actress,acting= act


In [56]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
pip install swifter

Defaulting to user installation because normal site-packages is not writeable
  Using cached swifter-1.4.0.tar.gz (1.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached dask-2024.8.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached partd-1.4.2-py3-none-any.whl.metadata (4.6 kB)
  Using cached toolz-1.0.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached dask_expr-1.1.10-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyarrow-19.0.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached locket-1.0.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached dask-2024.8.0-py3-none-any.whl (1.2 MB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached dask_expr-1.1.10-py3-none-any.whl (242 kB)
Using cached fsspec-2025.2.0-py3-none-any.whl (184 kB)
Using cached partd-1.4.2-py3-none-any.whl (18 kB)
Using cached toolz-1.0.0-py3-no

In [57]:
import re
import nltk
import pandas as pd
import swifter  # For fast parallel processing
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Download stopwords
nltk.download('stopwords')

# Load stopwords once (optimization)
stop_words = set(stopwords.words('english'))

# Initialize stemmer
port_stem = PorterStemmer()

# Optimized stemming function
def stemming(content):
    content = re.sub(r'[^a-zA-Z ]', '', content)  # Keep spaces
    words = content.lower().split()
    stemmed_words = [port_stem.stem(word) for word in words if word not in stop_words]
    return ' '.join(stemmed_words)

# Load dataset
file_path = "/Users/vishesh/Downloads/One8/training.1600000.processed.noemoticon.csv"
twitter_data = pd.read_csv(file_path, encoding="latin1", header=None)

# Assign column names (since dataset has no headers)
twitter_data.columns = ["target", "id", "date", "flag", "user", "text"]

# 🚀 Use swifter for super-fast parallel processing
twitter_data['stemmed_content'] = twitter_data['text'].swifter.apply(stemming)

# Save processed data (optional)
twitter_data.to_csv("processed_tweets.csv", index=False)

# Show first 5 rows
print(twitter_data.head())


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pandas Apply:   0%|          | 0/1600000 [00:00<?, ?it/s]

   target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   
3          ElleCTF    my whole body feels itchy and like its on fire    
4           Karoli  @nationwideclass no, it's not behaving at all....   

                                     stemmed_content  
0  switchfoot httptwitpiccomyzl awww that bummer ...  
1  upset cant updat facebook text might cri resul...  
2  ke

In [58]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww that bummer ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see


In [59]:
print(twitter_data['stemmed_content'])

0          switchfoot httptwitpiccomyzl awww that bummer ...
1          upset cant updat facebook text might cri resul...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                      nationwideclass behav im mad cant see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdbcom cool hear old walt interview httpbli...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [62]:
print(twitter_data.head())


   target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   
3          ElleCTF    my whole body feels itchy and like its on fire    
4           Karoli  @nationwideclass no, it's not behaving at all....   

                                     stemmed_content  
0  switchfoot httptwitpiccomyzl awww that bummer ...  
1  upset cant updat facebook text might cri resul...  
2  ke

In [66]:
# Ensure the column type is integer
twitter_data["target"] = twitter_data["target"].astype(int)

# Replace '4' with '1'
twitter_data["target"] = twitter_data["target"].replace(4, 1)

# Verify replacement
print(twitter_data["target"].value_counts())  # Should show only 0s and 1s


target
0    800000
1    800000
Name: count, dtype: int64


In [67]:
x= twitter_data['stemmed_content'].values
y= twitter_data['target'].values

In [68]:
print(x)

['switchfoot httptwitpiccomyzl awww that bummer shoulda got david carr third day'
 'upset cant updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguphh']


In [69]:
print(y)

[0 0 0 ... 1 1 1]


In [70]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, stratify=y,random_state=2)

In [71]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [72]:
#converting textutal data to numerical data
vectorizer= TfidfVectorizer()

x_train= vectorizer.fit_transform(x_train)
x_test= vectorizer.transform(x_test)

In [73]:
print(x_train)

  (0, 590524)	0.271935973896376
  (0, 485829)	0.35734734000828555
  (0, 275153)	0.5287163126006141
  (0, 135672)	0.3751210798745192
  (0, 330512)	0.4200753866446622
  (0, 599548)	0.4490363130629349
  (1, 199672)	0.9736525845879415
  (1, 265990)	0.2280364982304841
  (2, 135672)	0.46222653304912487
  (2, 265990)	0.11789035957962253
  (2, 153180)	0.1905056115706456
  (2, 552451)	0.1883319097978993
  (2, 160229)	0.29161611256480313
  (2, 551043)	0.3225385174402054
  (2, 586742)	0.3320039094486021
  (2, 96518)	0.3154389006079266
  (2, 600077)	0.33687662733057383
  (2, 368923)	0.24369409966007974
  (2, 555076)	0.1531113773352699
  (2, 186397)	0.1889237729533197
  (2, 163730)	0.20357992328484822
  (2, 395952)	0.16862711364982916
  (3, 551043)	0.28972941659572027
  (3, 197116)	0.4472150500940576
  (3, 188008)	0.2779040561459071
  :	:
  (1279996, 587294)	0.27112935728623483
  (1279996, 531652)	0.22006519052096826
  (1279996, 509536)	0.3605270658820192
  (1279996, 334168)	0.5223794704929838
  (1

In [75]:
print(x_test)

  (0, 18735)	0.17227271385168627
  (0, 39020)	0.15827445867713713
  (0, 84315)	0.2612587662159671
  (0, 131471)	0.35874567833011956
  (0, 163803)	0.24913841224739333
  (0, 171233)	0.23191862402987573
  (0, 199992)	0.2179563421051184
  (0, 258522)	0.27403042037763914
  (0, 374099)	0.440155313081504
  (0, 384931)	0.17345900492444394
  (0, 529152)	0.21456939414403495
  (0, 541819)	0.3387821615303071
  (0, 555076)	0.30718915166161176
  (0, 571715)	0.17566897667440953
  (1, 8008)	0.3053245680397504
  (1, 18735)	0.2144214773747336
  (1, 180088)	0.5700107365388162
  (1, 310429)	0.40607312375462945
  (1, 356802)	0.28423752828699317
  (1, 478367)	0.4807076997545237
  (1, 500333)	0.24314985399219477
  (2, 28150)	0.35334726711330017
  (2, 43324)	0.3799460828604977
  (2, 111216)	0.36316906406311267
  (2, 272498)	0.5865060038453336
  :	:
  (319994, 600596)	0.25994636406247995
  (319995, 133815)	0.37415114246535175
  (319995, 135835)	0.3387823265608489
  (319995, 192780)	0.31055261383164634
  (31999

In [76]:
#training machine learning model
model= LogisticRegression(max_iter=1000)

In [77]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [78]:
#accuracy score on the training data
x_train_prediction=model.predict(x_train)
training_data_accuracy= accuracy_score(y_train,x_train_prediction)
print('Accuracy score on the training data : ', training_data_accuracy)

Accuracy score on the training data :  0.81143125


In [79]:
x_test_prediction=model.predict(x_test)
test_data_accuracy= accuracy_score(y_test,x_test_prediction)
print('Accuracy score on the training data : ', test_data_accuracy)

Accuracy score on the training data :  0.782409375


In [80]:
import pickle

In [81]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [82]:
loaded_model= pickle.load(open('/Users/vishesh/Downloads/One8/trained_model.sav','rb'))

In [84]:
x_new= x_test[200]
print(y_test[200])

1


In [86]:
prediction= model.predict(x_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')
else:
    print('Positive tweet')

[1]
Positive tweet


In [87]:
x_new= x_test[3]
print(y_test[3])
prediction= model.predict(x_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')
else:
    print('Positive tweet')

0
[0]
Negative Tweet


In [100]:
import os

# Define folder structure
folders = ["static", "templates"]
files = {
    "app.py": "from flask import Flask\napp = Flask(__name__)\n@app.route('/')\ndef home():\n    return 'Hello, Flask!'\nif __name__ == '__main__':\n    app.run(debug=True)",
    "static/styles.css": "body { background-color: #f8f9fa; }",
    "templates/index.html": "<!DOCTYPE html>\n<html>\n<head>\n<title>Flask App</title>\n</head>\n<body>\n<h1>Welcome to Flask</h1>\n</body>\n</html>",
    "model.pkl": "",
    "requirements.txt": "flask\nnumpy\npickle5",
    "README.md": "# Flask ML Model Deployment\nThis is a Flask app to deploy a Machine Learning model."
}

# Create folders
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Create files with sample content
for file_path, content in files.items():
    with open(file_path, "w") as f:
        f.write(content)

print("Project structure created successfully! 🚀")


Project structure created successfully! 🚀


In [101]:
pip install flask pandas nltk scikit-learn swifter pickle5


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp39-cp39-macosx_10_9_universal2.whl size=169871 sha256=178f831ba903aa36988f56e8e8994437d0733b13543006a57ae712a4883d54ef
  Stored in directory: /Users/vishesh/Library/Caches/pip/wheels/f2/7a/49/9bef8878949914ecb90c08fc5bf30a05e17f475fe7e08b63a8
Successfully built pickle5

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [102]:
import pickle

# Save the vectorizer
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
print("Vectorizer saved successfully!")


Vectorizer saved successfully!
